In [1]:
from datetime import datetime, timedelta
import xarray as xr
import numpy as np
import torch
import yaml

import pandas as pd
from glob import glob
from os.path import join
import itertools
import os
import yaml

from credit.datasets.goes10km_dataset import ERA5Interpolator
from credit.datasets.era5 import ERA5Dataset

from joblib import Parallel, delayed
import joblib

from functools import partial

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
year = 2017
num_cpus = 2
test = 1

In [3]:
config = os.path.join("/glade/u/home/dkimpara/miles-credit/config/goes_era5_forcing.yml")

with open(config) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

target_times = pd.date_range(f"{year}-01-01",
                                    f"{year + 1}-01-01",
                                   freq="1h", inclusive="left")

def interp_era5(conf, target_times, time_subset):
    # build a xarray sampler for era5
    time_config = {
        "timestep": pd.Timedelta("1h"),
        "num_forecast_steps": 0,
        "start_datetime": target_times[0],
        "end_datetime": target_times[-1],
    }
    era5dataset = ERA5Dataset(conf, time_config, "ERA5")
    interp_dataset = ERA5Interpolator(conf["data"], era5dataset)

    for t in time_subset:
        interp_dataset[(t, None)]
def chunk_list(a, num_chunks):
    step = len(a) // num_chunks
    return [a[i:i + step] for i in range(0, len(a), step)]


In [4]:
%%time
if test:
    target_times = target_times[:10]

f = partial(interp_era5, conf, target_times)
if test:
    chunked = [target_times]
else:
    chunked = chunk_list(target_times, (num_cpus -1) * 10)    


results = Parallel(n_jobs = num_cpus - 1)(delayed(f)(time_list)
                            for time_list in chunked)


CPU times: user 1min 6s, sys: 14.4 s, total: 1min 21s
Wall time: 1min 30s
